# Evaluate RS

In [2]:
import psycopg2
import pandas as pd
import requests

In [3]:
def connect_and_query(query: str, params):
    conn = psycopg2.connect(
        host="localhost",
        port=5432,
        database="mp_dump",
        user="postgres",
        password="changeme"
    )

    cur = conn.cursor()

    cur.execute(query, params)
    res = cur.fetchall()

    cur.close()

    return res

In [34]:
def request_recommendations(viewed_service, purchase_history, metadata_weight, view_weight):
    data = {
        "service_id": viewed_service,
        "purchase_ids": purchase_history,
        "num": 5,
        "view_weight": view_weight,
        "metadata_weight": metadata_weight 
    }

    r = requests.post('http://0.0.0.0:4559/rs_evaluation/recommendation', json=data)
    
    return r.json()['service_ids']

In [15]:
EOSC_SERVICES_URL = "https://marketplace.eosc-portal.eu/services/"

def get_service_info(service_id):
    query = f"""
        SELECT id, name, slug
        FROM services
        WHERE id={service_id}
    """
    
    res = pd.DataFrame(connect_and_query(query, ()), columns=["id", "name", "slug"])
    res['slug'] = res['slug'].apply(lambda x: EOSC_SERVICES_URL + x)
    
    return res.to_dict()

In [37]:
def present_recommendations(viewed_service, purchase_history, metadata_weight=0.5, view_weight=0.5):
    viewed_df = get_service_info(viewed_service)
    purchase_history_df = [get_service_info(service_id) for service_id in purchase_history]
    
    recommended_ids = request_recommendations(viewed_service, purchase_history, metadata_weight, view_weight)
    recommended_df = [get_service_info(service_id) for service_id in recommended_ids]
    
    
    print("#"*180)
    print("> Currently viewing:")
    print(f"\t- {viewed_df['id'][0]} | {viewed_df['name'][0]} | {viewed_df['slug'][0]}")
    print()
    
    print("> Purchase history:")
    for service in purchase_history_df:
        print(f"\t- {service['id'][0]} | {service['name'][0]} | {service['slug'][0]}")
    print()
    
    print("> Recommendations:")
    for service in recommended_df:
        print(f"\t- {service['id'][0]} | {service['name'][0]} | {service['slug'][0]}")

## Experiments

**100 Percent IT Cloud**

In [38]:
present_recommendations(76, [], metadata_weight=0.5, view_weight=0.5)

####################################################################################################################################################################################
> Currently viewing:
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Purchase history:

> Recommendations:
	- 55 | ePouta Virtual Private Cloud | https://marketplace.eosc-portal.eu/services/csc-epouta
	- 177 | European Cloud Hosting | https://marketplace.eosc-portal.eu/services/european-cloud-hosting
	- 2 | EGI High-Throughput Compute | https://marketplace.eosc-portal.eu/services/egi-high-throughput-compute
	- 1 | EGI Cloud Compute | https://marketplace.eosc-portal.eu/services/egi-cloud-compute
	- 69 | Embassy Cloud | https://marketplace.eosc-portal.eu/services/embassy-cloud


In [23]:
present_recommendations(76, [], metadata_weight=1)

# We can see services that do not relate with the service we currently view EGI Check-In, Open Knowledge Maps

####################################################################################################################################################################################
> Currently viewing:
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Purchase history:

> Recommendations:
	- 55 | ePouta Virtual Private Cloud | https://marketplace.eosc-portal.eu/services/csc-epouta
	- 2 | EGI High-Throughput Compute | https://marketplace.eosc-portal.eu/services/egi-high-throughput-compute
	- 111 | Rahti Container Cloud | https://marketplace.eosc-portal.eu/services/rahti-container-cloud
	- 45 | EGI Check-In | https://marketplace.eosc-portal.eu/services/egi-check-in
	- 362 | Open Knowledge Maps | https://marketplace.eosc-portal.eu/services/open-knowledge-maps


In [29]:
present_recommendations(76, [], metadata_weight=0)

# Similar to metada_weight=0.5

####################################################################################################################################################################################
> Currently viewing:
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Purchase history:

> Recommendations:
	- 511 | CLOUDIFIN | https://marketplace.eosc-portal.eu/services/cloudifin
	- 1 | EGI Cloud Compute | https://marketplace.eosc-portal.eu/services/egi-cloud-compute
	- 79 | MetaCentrum Cloud | https://marketplace.eosc-portal.eu/services/metacentrum-cloud
	- 227 | Open Telekom Cloud | https://marketplace.eosc-portal.eu/services/open-telekom-cloud
	- 622 | SCIGNE Cloud Compute | https://marketplace.eosc-portal.eu/services/scigne-cloud-compute


In [28]:
present_recommendations(76, [419], metadata_weight=0.5)

# Interesting recommendation: de.NBI Cloud: Cloud Computing for Life Sciences which combines both viewing and history

####################################################################################################################################################################################
> Currently viewing:
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Purchase history:
	- 419 | DisVis web portal | https://marketplace.eosc-portal.eu/services/disvis-web-portal-6eab178c-9bc5-4c62-b7ce-aeeb18d5cba9

> Recommendations:
	- 69 | Embassy Cloud | https://marketplace.eosc-portal.eu/services/embassy-cloud
	- 177 | European Cloud Hosting | https://marketplace.eosc-portal.eu/services/european-cloud-hosting
	- 363 | de.NBI Cloud: Cloud Computing for Life Sciences | https://marketplace.eosc-portal.eu/services/cloud-computing-for-life-sciences
	- 425 | PDB-Tools web | https://marketplace.eosc-portal.eu/services/pdb-tools-web
	- 42 | CloudFerro Infrastructure | https://marketplace.eosc-portal.eu/services/cloudferro-infrastructure


In [33]:
present_recommendations(76, [419], metadata_weight=1)

# Focuses on web portals and offers 1 cloud solution

####################################################################################################################################################################################
> Currently viewing:
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Purchase history:
	- 419 | DisVis web portal | https://marketplace.eosc-portal.eu/services/disvis-web-portal-6eab178c-9bc5-4c62-b7ce-aeeb18d5cba9

> Recommendations:
	- 420 | HADDOCK2.4 web portal | https://marketplace.eosc-portal.eu/services/haddock2-4-web-portal
	- 421 | PowerFit web portal | https://marketplace.eosc-portal.eu/services/powerfit-web-portal-b8ddee6c-78f5-43d8-a5a2-9e3b7f1cb24e
	- 425 | PDB-Tools web | https://marketplace.eosc-portal.eu/services/pdb-tools-web
	- 422 | SpotOn web portal | https://marketplace.eosc-portal.eu/services/spoton-c5db8fd5-a546-4342-8bae-2b2b4777b67e
	- 69 | Embassy Cloud | https://marketplace.eosc-portal.eu/services/embassy-cloud


In [43]:
present_recommendations(76, [419], metadata_weight=0.5, view_weight=0.8)

####################################################################################################################################################################################
> Currently viewing:
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Purchase history:
	- 419 | DisVis web portal | https://marketplace.eosc-portal.eu/services/disvis-web-portal-6eab178c-9bc5-4c62-b7ce-aeeb18d5cba9

> Recommendations:
	- 177 | European Cloud Hosting | https://marketplace.eosc-portal.eu/services/european-cloud-hosting
	- 69 | Embassy Cloud | https://marketplace.eosc-portal.eu/services/embassy-cloud
	- 55 | ePouta Virtual Private Cloud | https://marketplace.eosc-portal.eu/services/csc-epouta
	- 2 | EGI High-Throughput Compute | https://marketplace.eosc-portal.eu/services/egi-high-throughput-compute
	- 1 | EGI Cloud Compute | https://marketplace.eosc-portal.eu/services/egi-cloud-compute


In [48]:
present_recommendations(76, [419, 352], metadata_weight=0.5, view_weight=0.5)

####################################################################################################################################################################################
> Currently viewing:
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Purchase history:
	- 419 | DisVis web portal | https://marketplace.eosc-portal.eu/services/disvis-web-portal-6eab178c-9bc5-4c62-b7ce-aeeb18d5cba9
	- 352 | E-Learning Platform of GBIF Spain | https://marketplace.eosc-portal.eu/services/e-learning-platform-of-gbif-spain

> Recommendations:
	- 511 | CLOUDIFIN | https://marketplace.eosc-portal.eu/services/cloudifin
	- 363 | de.NBI Cloud: Cloud Computing for Life Sciences | https://marketplace.eosc-portal.eu/services/cloud-computing-for-life-sciences
	- 79 | MetaCentrum Cloud | https://marketplace.eosc-portal.eu/services/metacentrum-cloud
	- 177 | European Cloud Hosting | https://marketplace.eosc-portal.eu/services/european-cloud

In [49]:
present_recommendations(352, [419, 76], metadata_weight=0.5, view_weight=0.5)

####################################################################################################################################################################################
> Currently viewing:
	- 352 | E-Learning Platform of GBIF Spain | https://marketplace.eosc-portal.eu/services/e-learning-platform-of-gbif-spain

> Purchase history:
	- 419 | DisVis web portal | https://marketplace.eosc-portal.eu/services/disvis-web-portal-6eab178c-9bc5-4c62-b7ce-aeeb18d5cba9
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Recommendations:
	- 363 | de.NBI Cloud: Cloud Computing for Life Sciences | https://marketplace.eosc-portal.eu/services/cloud-computing-for-life-sciences
	- 210 | Software Integration Support | https://marketplace.eosc-portal.eu/services/software-integration-support
	- 51 | PaN faas | https://marketplace.eosc-portal.eu/services/pan-faas
	- 511 | CLOUDIFIN | https://marketplace.eosc-portal.eu/services/cloudif

**KER - Keyword Extractor**

In [55]:
present_recommendations(496, [], metadata_weight=0.5, view_weight=0.5)
# Proposes similar solutions from the same team

####################################################################################################################################################################################
> Currently viewing:
	- 496 | KER - Keyword Extractor | https://marketplace.eosc-portal.eu/services/ker-keyword-extractor

> Purchase history:

> Recommendations:
	- 493 | MorphoDiTa | https://marketplace.eosc-portal.eu/services/morphodita
	- 495 | Machine Translation | https://marketplace.eosc-portal.eu/services/machine-translation
	- 492 | ElixirFM | https://marketplace.eosc-portal.eu/services/elixirfm
	- 494 | NameTag | https://marketplace.eosc-portal.eu/services/nametag
	- 434 | UDPipe: tool for lemmatization, morphological analysis, POS tagging and dependency parsing in multiple languages | https://marketplace.eosc-portal.eu/services/udpipe-tool-for-lemmatization-morphological-analysis-pos-tagging-and-dependency-parsing-in-multiple-languages


In [57]:
present_recommendations(496, [], metadata_weight=0, view_weight=0.5)
# While the first two are related (with the first one being from a diff organisation) the rest of them are not

####################################################################################################################################################################################
> Currently viewing:
	- 496 | KER - Keyword Extractor | https://marketplace.eosc-portal.eu/services/ker-keyword-extractor

> Purchase history:

> Recommendations:
	- 626 | The Tromsø Repository of Language and Linguistics (TROLLing) | https://marketplace.eosc-portal.eu/services/the-tromso-repository-of-language-and-linguistics-trolling
	- 434 | UDPipe: tool for lemmatization, morphological analysis, POS tagging and dependency parsing in multiple languages | https://marketplace.eosc-portal.eu/services/udpipe-tool-for-lemmatization-morphological-analysis-pos-tagging-and-dependency-parsing-in-multiple-languages
	- 145 | V-SEEM CLOWDER | https://marketplace.eosc-portal.eu/services/v-seem-clowder
	- 424 | AMBER-based Portal Server for NMR structures (AMPS-NMR) | https://marketplace.eosc-portal.eu/services/amber-b

In [62]:
present_recommendations(496, [17], metadata_weight=0.5, view_weight=0.5)
# Virtual Language Observatory relates with the purchase history

####################################################################################################################################################################################
> Currently viewing:
	- 496 | KER - Keyword Extractor | https://marketplace.eosc-portal.eu/services/ker-keyword-extractor

> Purchase history:
	- 17 | Language Resource Switchboard | https://marketplace.eosc-portal.eu/services/language-resource-switchboard

> Recommendations:
	- 495 | Machine Translation | https://marketplace.eosc-portal.eu/services/machine-translation
	- 493 | MorphoDiTa | https://marketplace.eosc-portal.eu/services/morphodita
	- 18 | Virtual Collection Registry | https://marketplace.eosc-portal.eu/services/virtual-collection-registry
	- 492 | ElixirFM | https://marketplace.eosc-portal.eu/services/elixirfm
	- 16 | Virtual Language Observatory | https://marketplace.eosc-portal.eu/services/virtual-language-observatory
